In [24]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [25]:
!pip install --upgrade --force-reinstall scipy


  Using cached scipy-1.15.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached scipy-1.15.3-cp311-cp311-win_amd64.whl (41.2 MB)
Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl (12.9 MB)

  Attempting uninstall: numpy

    Found existing installation: numpy 2.2.6

   ---------------------------------------- 0/2 [numpy]
    Uninstalling numpy-2.2.6:
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
      Successfully uninstalled numpy-2.2.6
   ----------------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow-intel 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 2.2.6 which is incompatible.
tensorflow-intel 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.19.0 which is incompatible.


In [26]:
# load the ann tranied model, scaler pickle, onehot
model=load_model('model.h5')

# load the encoder and scaler

with open('label_encode_gender.pkl','rb') as file:
    label_encode_gender=pickle.load(file)
    
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)



with open('scaler.pkl','rb')as file:
    scaler=pickle.load(file)

In [27]:
# example input data
input_data={
    "CreditScore": 600,
    "Geography": 'France',
    "Gender": 'Male',
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}

In [28]:
# one hot encode 'Geography' 
geo_encoded=label_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df=pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\RESHMA\Desktop\ANN classification\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [29]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [30]:
# encode categorical variables
input_df['Gender']=label_encode_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [31]:
# concatination one hot encoded
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [33]:
# scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled


array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [34]:
# predict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 351ms/step


array([[0.03038218]], dtype=float32)

In [36]:
prediction_proba=prediction[0][0]
prediction

array([[0.03038218]], dtype=float32)

In [37]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')    

The customer is not likely to churn.
